# Advanced Visualization

This notebook demonstrates the visualization capabilities of `rademacher-backtest`.

**Note**: This requires the `[viz]` extra to be installed:

```bash
pip install rademacher-backtest[viz]
```

This adds matplotlib and seaborn for professional-quality charts.

## Setup and Run Backtest

In [ ]:
import pandas as pd
import numpy as np
import rademacher_backtest as rbt
import matplotlib.pyplot as plt
import seaborn as sns

# Set style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

# Load data and run backtest
prices = pd.read_csv('data/sample_prices.csv', index_col='date', parse_dates=True)
loader = rbt.DataFrameLoader(prices)

result = rbt.backtest(
    portfolio={'SPY': 0.60, 'AGG': 0.40},
    loader=loader,
    start_date='2015-01-01',
    end_date='2023-12-31',
)

print(f"✅ Backtest complete: {len(result.daily_returns)} days")

## 1. Cumulative Returns Chart

The classic cumulative returns visualization:

In [ ]:
# Create charts object
charts = rbt.BacktestCharts()

# Plot cumulative returns
fig = charts.cumulative_returns(result.daily_values)
plt.show()

## 2. Drawdown Analysis

Visualize the peak-to-trough drawdowns over time:

In [ ]:
# Plot drawdown
fig = charts.drawdown(result.daily_returns)
plt.show()

## 3. Monthly Returns Heatmap

A calendar heatmap showing monthly returns:

In [ ]:
# Create heatmap
heatmap = rbt.ReturnHeatmap()
fig = heatmap.plot(result.daily_returns)
plt.show()

## 4. RAS Haircut Decomposition

Visualize how RAS adjusts the Sharpe ratio:

In [ ]:
# Perform RAS analysis
ras_result = rbt.analyze_ras(result.daily_returns, confidence=0.99)

# Plot RAS decomposition
ras_viz = rbt.RASVisualization()
fig = ras_viz.haircut_decomposition(ras_result)
plt.show()

## 5. Returns Distribution

Analyze the distribution of daily returns:

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Histogram
axes[0].hist(result.daily_returns * 100, bins=50, alpha=0.7, edgecolor='black')
axes[0].axvline(result.daily_returns.mean() * 100, color='red', 
               linestyle='--', linewidth=2, label='Mean')
axes[0].set_xlabel('Daily Return (%)', fontsize=11)
axes[0].set_ylabel('Frequency', fontsize=11)
axes[0].set_title('Distribution of Daily Returns', fontsize=12, fontweight='bold')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Q-Q plot
from scipy import stats
stats.probplot(result.daily_returns, dist="norm", plot=axes[1])
axes[1].set_title('Q-Q Plot (Normal Distribution)', fontsize=12, fontweight='bold')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Statistics
print("\n📊 Returns Statistics:")
print(f"  Mean: {result.daily_returns.mean() * 100:.4f}%")
print(f"  Std Dev: {result.daily_returns.std() * 100:.4f}%")
print(f"  Skewness: {result.daily_returns.skew():.4f}")
print(f"  Kurtosis: {result.daily_returns.kurtosis():.4f}")
print(f"  Min: {result.daily_returns.min() * 100:.4f}%")
print(f"  Max: {result.daily_returns.max() * 100:.4f}%")

## 6. Rolling Sharpe Ratio

Track how the Sharpe ratio evolves over time:

In [ ]:
# Calculate rolling Sharpe ratio (252-day window)
window = 252  # 1 year
rolling_mean = result.daily_returns.rolling(window=window).mean()
rolling_std = result.daily_returns.rolling(window=window).std()
rolling_sharpe = (rolling_mean / rolling_std) * np.sqrt(252)

# Calculate performance metrics for final Sharpe
perf = rbt.PerformanceCalculator().calculate(result.daily_returns)

plt.figure(figsize=(14, 6))
plt.plot(rolling_sharpe.index, rolling_sharpe.values, linewidth=2, label='Rolling Sharpe (1Y)')
plt.axhline(perf.sharpe_ratio, color='green', linestyle='--', 
           linewidth=2, label=f'Full Period Sharpe: {perf.sharpe_ratio:.3f}')
plt.axhline(ras_result.adjusted_sharpe_annualized, color='red', linestyle='--', 
           linewidth=2, label=f'RAS-Adjusted: {ras_result.adjusted_sharpe_annualized:.3f}')
plt.axhline(0, color='black', linestyle='-', linewidth=1, alpha=0.5)
plt.xlabel('Date', fontsize=12)
plt.ylabel('Sharpe Ratio (Annualized)', fontsize=12)
plt.title('Rolling Sharpe Ratio (252-Day Window)', fontsize=14, fontweight='bold')
plt.legend(loc='best')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 7. Underwater Plot (Drawdown Duration)

Visualize how long it takes to recover from drawdowns:

In [ ]:
# Calculate cumulative returns and running maximum
cumulative = (1 + result.daily_returns).cumprod()
running_max = cumulative.expanding().max()
underwater = (cumulative / running_max - 1) * 100

plt.figure(figsize=(14, 6))
plt.fill_between(underwater.index, underwater.values, 0, 
                alpha=0.5, color='red', label='Drawdown')
plt.plot(underwater.index, underwater.values, color='darkred', linewidth=1.5)
plt.xlabel('Date', fontsize=12)
plt.ylabel('Drawdown (%)', fontsize=12)
plt.title('Underwater Plot - Time Below Peak', fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3)
plt.legend()
plt.tight_layout()
plt.show()

print(f"\n📉 Drawdown Analysis:")
print(f"  Maximum Drawdown: {underwater.min():.2f}%")
print(f"  Current Drawdown: {underwater.iloc[-1]:.2f}%")

# Find longest drawdown
is_underwater = underwater < -0.01  # More than 0.01% below peak
if is_underwater.any():
    # Find sequences of underwater periods
    underwater_periods = is_underwater.astype(int).diff().fillna(0)
    print(f"  Days underwater: {is_underwater.sum()} / {len(underwater)} ({100*is_underwater.sum()/len(underwater):.1f}%)")

## 8. Save All Charts

Export all charts to files:

In [ ]:
import os

# Create output directory
output_dir = 'output/charts'
os.makedirs(output_dir, exist_ok=True)

# Generate and save all charts
print("Generating charts...")

# 1. Cumulative returns
fig = charts.cumulative_returns(result.daily_values)
charts.save_figure(fig, f'{output_dir}/cumulative_returns.png')
print("  ✓ Cumulative returns")

# 2. Drawdown
fig = charts.drawdown(result.daily_returns)
charts.save_figure(fig, f'{output_dir}/drawdown.png')
print("  ✓ Drawdown")

# 3. Monthly heatmap
fig = heatmap.plot(result.daily_returns)
charts.save_figure(fig, f'{output_dir}/monthly_heatmap.png')
print("  ✓ Monthly heatmap")

# 4. RAS decomposition
fig = ras_viz.haircut_decomposition(ras_result)
charts.save_figure(fig, f'{output_dir}/ras_decomposition.png')
print("  ✓ RAS decomposition")

print(f"\n✅ All charts saved to {output_dir}/")

## Summary

This notebook demonstrated the visualization capabilities:

1. ✅ **Cumulative Returns** - Track portfolio value over time
2. ✅ **Drawdown Chart** - Identify peak-to-trough declines
3. ✅ **Monthly Heatmap** - Calendar view of returns
4. ✅ **RAS Decomposition** - Understand statistical adjustments
5. ✅ **Returns Distribution** - Analyze return characteristics
6. ✅ **Rolling Sharpe** - Monitor risk-adjusted performance
7. ✅ **Underwater Plot** - Track recovery from drawdowns
8. ✅ **Export to Files** - Save charts for reports

## Custom Styling

All charts can be customized using matplotlib/seaborn:

```python
# Set custom style
sns.set_style("darkgrid")
plt.rcParams['figure.figsize'] = (16, 8)
plt.rcParams['font.size'] = 12

# Generate chart
fig = charts.cumulative_returns(result.daily_values)

# Customize further
ax = fig.gca()
ax.set_facecolor('#f0f0f0')
plt.show()
```

## Integration with Reports

Charts can be easily integrated into:
- PDF reports
- PowerPoint presentations
- Web dashboards
- Research papers

All figures are publication-quality and use professional styling.